In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import nolds

from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf
from mne.channels import make_standard_montage


from scipy.signal import detrend
from tqdm import tqdm
from scipy.stats import linregress

from mne.time_frequency import tfr_array_morlet
from scipy.signal import hilbert, butter, filtfilt


In [ ]:
import my_functions2 as mf

In [ ]:
subject = 1
runs = {
    "rest": [1],
    "motor_execution": [3, 7, 11],
    "motor_imagery": [4, 8, 12]
}

# Load and preprocess the data
subject_data = mf.load_and_preprocess_subject(subject, runs)

In [ ]:
%matplotlib qt

# Quick visualization check
mf.quick_plot(subject_data["rest"], title=f"Subject {subject} | REST")
mf.quick_plot(subject_data["motor_execution"], title=f"Subject {subject} | MOTOR EXECUTION")

In [ ]:
# Extract epochs from each condition
epochs_rest = mf.extract_clean_epochs(subject_data["rest"])
epochs_exec = mf.extract_clean_epochs(subject_data["motor_execution"]) 
epochs_imag = mf.extract_clean_epochs(subject_data["motor_imagery"])

In [ ]:
# Combine epochs for more robust analysis
combined_rest = mne.concatenate_epochs([
    epochs_exec['rest'], 
    epochs_imag['rest'], 
    epochs_rest['rest']
])

combined_task = mne.concatenate_epochs([
    epochs_exec['task'], 
    epochs_imag['task']
])

# Shuffle task epochs for fair sampling
np.random.seed(42)  # for reproducibility
shuffled_indices = np.random.permutation(len(combined_task))
combined_task = combined_task[shuffled_indices]

print(f"Combined REST epochs: {len(combined_rest)}")
print(f"Combined TASK epochs: {len(combined_task)}")

In [ ]:
%matplotlib qt 
combined_task.plot(n_channels=8, title="Combined Task Epochs")

# DFA Analysis

In [ ]:
# DFA ON RAW SIGNAL

In [ ]:
# Define channels of interest and fitting range
channels = ['C3', 'Cz', 'C4']
# channels = ['C3', 'C4', 'CP3', 'CP4', 'Cz', 'CPz', 'FC3', 'FC4', 'O1', 'O2']
fit_range = (50, 5000)  # in samples

# Compute DFA on raw signal for both conditions
dfa_raw_rest = mf.compute_dfa_from_epochs(combined_rest, picks=channels, fit_range=fit_range)
dfa_raw_task = mf.compute_dfa_from_epochs(combined_task, picks=channels, fit_range=fit_range)

# Print results
print("\nRaw Signal DFA Results:")
print("-" * 30)
print("Channel | Rest Alpha | Task Alpha")
print("-" * 30)
for ch in channels:
    print(f"{ch:7} | {dfa_raw_rest[ch]:.3f} | {dfa_raw_task[ch]:.3f}")

# Visual comparison for channel Cz
mf.compare_conditions_dfa(
    {'Rest': combined_rest, 'Task': combined_task}, 
    channel='Cz', 
    fit_range=fit_range
)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def plot_dfa_comparison_bars(dfa_rest, dfa_task, channels=None, title="DFA Comparison (Rest vs Task)", figsize=(12, 6), show_diff=True):
    """
    Plots a bar chart comparing DFA alpha exponents for each channel between rest and task.

    Parameters:
    -----------
    dfa_rest : dict
        DFA alpha values for rest condition (channel -> alpha)
    dfa_task : dict
        DFA alpha values for task condition (channel -> alpha)
    channels : list or None
        List of channel names to plot (if None, uses intersection of both)
    title : str
        Plot title
    figsize : tuple
        Size of the matplotlib figure
    show_diff : bool
        Whether to show task - rest as text above bars
    """
    # Default: use common channels
    if channels is None:
        channels = sorted(set(dfa_rest.keys()) & set(dfa_task.keys()))

    # Build dataframe for plotting
    data = []
    for ch in channels:
        rest_alpha = dfa_rest[ch]
        task_alpha = dfa_task[ch]
        data.append({"Channel": ch, "Condition": "Rest", "Alpha": rest_alpha})
        data.append({"Channel": ch, "Condition": "Task", "Alpha": task_alpha})

    df = pd.DataFrame(data)

    # Plot
    plt.figure(figsize=figsize)
    ax = sns.barplot(data=df, x="Channel", y="Alpha", hue="Condition", palette="Set2")

    # Optionally show differences
    if show_diff:
        for i, ch in enumerate(channels):
            rest_val = dfa_rest[ch]
            task_val = dfa_task[ch]
            diff = task_val - rest_val
            max_val = max(rest_val, task_val)
            ax.text(i, max_val + 0.01, f"Δ={diff:+.3f}", ha='center', fontsize=9, color='black')

    # Labels and formatting
    plt.title(title)
    plt.ylabel("DFA Scaling Exponent (α)")
    plt.ylim(min(df["Alpha"]) - 0.05, max(df["Alpha"]) + 0.08)
    plt.grid(axis='y', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_dfa_comparison_bars(dfa_raw_rest, dfa_raw_task, channels=channels)

In [ ]:
# DFA ON FREQUENCY BANDS

In [ ]:
# Define frequency bands
bands = {
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 45)
}

# Analyze each band for each condition
for band_name, freq_range in bands.items():
    print(f"\n{band_name.upper()} BAND ({freq_range[0]}-{freq_range[1]} Hz)")
    print("-" * 50)
    
    # Compare conditions
    results = mf.compare_conditions_dfa(
        {'Rest': combined_rest, 'Task': combined_task},
        channel='Cz',
        band=freq_range,
        fit_range=(30, 1000)  # Adjusted for envelope signal
    )

In [ ]:
# DETAILED ANALYSIS FOR BETA BAND (13-30 Hz)

In [ ]:
# Analyze beta band across all channels
channels_extended = ['C3', 'Cz', 'C4', 'FC3', 'FCz', 'FC4', 'CP3', 'CPz', 'CP4']
beta_band = (13, 30)

# Compute DFA on beta envelope for all channels
dfa_beta_rest = mf.compute_dfa_from_epochs(
    combined_rest, 
    picks=channels_extended, 
    band=beta_band, 
    fit_range=(30, 1000)
)

dfa_beta_task = mf.compute_dfa_from_epochs(
    combined_task, 
    picks=channels_extended, 
    band=beta_band, 
    fit_range=(30, 1000)
)

# Create a comparison table
print("\nBeta Band DFA Results:")
print("-" * 40)
print("Channel | Rest Alpha | Task Alpha | Difference")
print("-" * 40)
for ch in channels_extended:
    diff = dfa_beta_task[ch] - dfa_beta_rest[ch]
    print(f"{ch:7} | {dfa_beta_rest[ch]:.3f} | {dfa_beta_task[ch]:.3f} | {diff:+.3f}")

# Simplified visualization of beta DFA results
plt.figure(figsize=(10, 6))

# Prepare data for plotting
channels = np.array(channels_extended)
rest_values = np.array([dfa_beta_rest[ch] for ch in channels])
task_values = np.array([dfa_beta_task[ch] for ch in channels])
diff_values = task_values - rest_values

# Sort channels by difference for better visualization
sort_idx = np.argsort(diff_values)
channels_sorted = channels[sort_idx]
rest_sorted = rest_values[sort_idx]
task_sorted = task_values[sort_idx]
diff_sorted = diff_values[sort_idx]

# Bar plot of DFA values by channel
x = np.arange(len(channels))
width = 0.35

plt.bar(x - width/2, rest_sorted, width, label='Rest', color='steelblue')
plt.bar(x + width/2, task_sorted, width, label='Task', color='firebrick')

# Add labels and formatting
plt.xlabel('EEG Channel')
plt.ylabel('DFA α Exponent')
plt.title('Beta Band (13-30 Hz) DFA Values by Channel and Condition')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(x, channels_sorted, rotation=45)
plt.ylim(0.65, 0.8)  # Adjust based on your data range
plt.legend()
plt.tight_layout()
plt.show()

# Plot the differences separately
plt.figure(figsize=(10, 4))
colors = ['firebrick' if val > 0 else 'steelblue' for val in diff_sorted]
plt.bar(channels_sorted, diff_sorted, color=colors)
plt.axhline(y=0, color='k', linestyle='-', alpha=0.5)
plt.xlabel('EEG Channel')
plt.ylabel('DFA Difference (Task - Rest)')
plt.title('Changes in Beta Band DFA: Task vs Rest')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
print("\n" + "="*80)
print("SUMMARY & DISCUSSION")
print("="*80)

# Calculate overall mean DFA values
mean_raw_rest = np.mean([dfa_raw_rest[ch] for ch in channels])
mean_raw_task = np.mean([dfa_raw_task[ch] for ch in channels])
mean_beta_rest = np.mean([dfa_beta_rest[ch] for ch in channels_extended])
mean_beta_task = np.mean([dfa_beta_task[ch] for ch in channels_extended])

print(f"Mean Raw DFA - Rest: {mean_raw_rest:.3f}")
print(f"Mean Raw DFA - Task: {mean_raw_task:.3f}")
print(f"Mean Beta DFA - Rest: {mean_beta_rest:.3f}")
print(f"Mean Beta DFA - Task: {mean_beta_task:.3f}")

# Discuss findings
print("\nINTERPRETATION:")
print("-" * 50)
print("DFA α values interpretation:")
print("- α < 0.5: Anti-persistent (negatively correlated)")
print("- α ≈ 0.5: White noise (uncorrelated)")
print("- 0.5 < α < 1.0: Persistent long-range correlations")
print("- α ≈ 1.0: 1/f noise (pink noise)")
print("- α > 1.0: Non-stationary, unbounded")

# Check for significant differences
if abs(mean_beta_task - mean_beta_rest) > 0.1:
    print("\nSignificant difference detected in beta band DFA between rest and task.")
    if mean_beta_task > mean_beta_rest:
        print("Task condition shows more persistent correlations (more complexity).")
    else:
        print("Rest condition shows more persistent correlations (more complexity).")
else:
    print("\nNo major differences detected in DFA scaling between rest and task.")

# Multiple Subject Analysis

In [ ]:
def compute_subject_dfa(subject_id, channels, band=None, fit_range=(50, 5000), envelope_fit_range=None):
    """
    Compute DFA for a single subject.
    
    Parameters:
    -----------
    subject_id : int
        Subject number (e.g., 1)
    channels : list
        EEG channels to include
    band : tuple or None
        Frequency band for envelope DFA (None for raw signal)
    fit_range : tuple
        DFA fit range in samples (raw)
    envelope_fit_range : tuple
        DFA fit range for band envelopes

    Returns:
    --------
    df_subject : DataFrame
        DataFrame with columns: Subject, Channel, Condition, Alpha
    """
    runs = {
        "rest": [1],
        "motor_execution": [3, 7, 11],
        "motor_imagery": [4, 8, 12]
    }

    # Load & preprocess
    subject_data = mf.load_and_preprocess_subject(subject_id, runs)

    # Extract epochs
    epochs_rest = mf.extract_clean_epochs(subject_data["rest"])
    epochs_exec = mf.extract_clean_epochs(subject_data["motor_execution"]) 
    epochs_imag = mf.extract_clean_epochs(subject_data["motor_imagery"])

    # Combine
    combined_rest = mne.concatenate_epochs([
        epochs_exec['rest'], epochs_imag['rest'], epochs_rest['rest']
    ])
    combined_task = mne.concatenate_epochs([
        epochs_exec['task'], epochs_imag['task']
    ])
    np.random.seed(42)
    combined_task = combined_task[np.random.permutation(len(combined_task))]

    # Choose DFA method
    if band is None:
        dfa_rest = mf.compute_dfa_from_epochs(combined_rest, picks=channels, fit_range=fit_range)
        dfa_task = mf.compute_dfa_from_epochs(combined_task, picks=channels, fit_range=fit_range)
    else:
        dfa_rest = mf.compute_dfa_from_epochs(combined_rest, picks=channels, band=band, fit_range=envelope_fit_range)
        dfa_task = mf.compute_dfa_from_epochs(combined_task, picks=channels, band=band, fit_range=envelope_fit_range)

    # Structure into DataFrame
    rows = []
    for ch in channels:
        rows.append({"Subject": subject_id, "Channel": ch, "Condition": "Rest", "Alpha": dfa_rest[ch]})
        rows.append({"Subject": subject_id, "Channel": ch, "Condition": "Task", "Alpha": dfa_task[ch]})
    
    return pd.DataFrame(rows)

In [ ]:
all_subjects = range(1, 11)  # Adjust as needed
channels = ['C3', 'Cz', 'C4']
beta_band = (13, 30)

df_all_dfa = pd.concat([
    compute_subject_dfa(subj, channels, band=beta_band, envelope_fit_range=(30, 1000))
    for subj in all_subjects
], ignore_index=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_group_dfa(df, title="Group DFA Comparison"):
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df, x="Channel", y="Alpha", hue="Condition", ci="sd", capsize=0.1)
    plt.title(title)
    plt.ylabel("DFA α Exponent")
    plt.grid(axis='y', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
from scipy.stats import ttest_rel

def paired_ttest_dfa(df):
    channels = df["Channel"].unique()
    results = []

    for ch in channels:
        df_ch = df[df["Channel"] == ch]
        df_pivot = df_ch.pivot(index="Subject", columns="Condition", values="Alpha").dropna()
        t_stat, p_val = ttest_rel(df_pivot["Task"], df_pivot["Rest"])
        results.append({"Channel": ch, "t": t_stat, "p": p_val})
    
    return pd.DataFrame(results)


In [ ]:
plot_group_dfa(df_all_dfa, title="Group DFA Comparison (Beta Band)")

In [ ]:
paired_ttest_dfa(df_all_dfa)

In [ ]:
bands = {
    "alpha": (8, 13),
    "beta": (13, 30)
}

df_all_bands = []

for band_name, band_range in bands.items():
    df_band = pd.concat([
        compute_subject_dfa(subj, channels, band=band_range, envelope_fit_range=(30, 1000))
        .assign(Band=band_name)
        for subj in all_subjects
    ])
    df_all_bands.append(df_band)

df_all_bands = pd.concat(df_all_bands, ignore_index=True)


In [ ]:
plot_group_dfa(df_all_bands, title="Group DFA Comparison (All Bands)")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_dfa_grouped_by_band(df, figsize=(14, 6), title="DFA by Channel, Condition & Band"):
    """
    Creates a grouped barplot for DFA results across bands and conditions.
    
    Parameters:
    -----------
    df : DataFrame
        Must contain columns: 'Channel', 'Condition', 'Alpha', 'Band'
    figsize : tuple
        Size of the plot
    title : str
        Plot title
    """
    plt.figure(figsize=figsize)
    
    sns.barplot(
        data=df,
        x="Channel",
        y="Alpha",
        hue="Condition",
        palette="Set2",
        ci="sd",
        capsize=0.1,
        dodge=True,
        errorbar="ci"
    )

    plt.title(title)
    plt.ylabel("DFA α Exponent")
    plt.grid(axis='y', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
for band in df_all_bands["Band"].unique():
    df_band = df_all_bands[df_all_bands["Band"] == band]
    plot_dfa_grouped_by_band(df_band, title=f"{band.capitalize()} Band DFA Comparison")

In [ ]:
def plot_dfa_facet_by_band(df, figsize=(14, 6)):
    """
    Faceted seaborn barplot: one subplot per band.
    """
    g = sns.catplot(
        data=df,
        kind="bar",
        x="Channel",
        y="Alpha",
        hue="Condition",
        col="Band",
        palette="Set2",
        ci="sd",
        capsize=0.1,
        height=figsize[1],
        aspect=figsize[0] / figsize[1]
    )
    g.set_titles("{col_name} Band")
    g.set_axis_labels("Channel", "DFA α Exponent")
    for ax in g.axes.flat:
        ax.grid(axis='y', linestyle='--', alpha=0.3)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_dfa_facet_by_band(df_all_bands)

# Real vs Imagined

In [ ]:
combined_rest = mne.concatenate_epochs([
    epochs_exec['rest'], epochs_imag['rest'], epochs_rest['rest']
])

combined_real = epochs_exec['task']
combined_imagined = epochs_imag['task']

In [ ]:
dfa_rest = mf.compute_dfa_from_epochs(combined_rest, picks=channels)
dfa_real = mf.compute_dfa_from_epochs(combined_real, picks=channels)
dfa_imag = mf.compute_dfa_from_epochs(combined_imagined, picks=channels)

In [ ]:
def compute_subject_dfa(subject_id, channels, band=None, fit_range=(50, 5000), envelope_fit_range=None):
    ...
    combined_rest = mne.concatenate_epochs([...])
    combined_real = epochs_exec['task']
    combined_imag = epochs_imag['task']

    if band is None:
        dfa_rest = mf.compute_dfa_from_epochs(combined_rest, picks=channels, fit_range=fit_range)
        dfa_real = mf.compute_dfa_from_epochs(combined_real, picks=channels, fit_range=fit_range)
        dfa_imag = mf.compute_dfa_from_epochs(combined_imag, picks=channels, fit_range=fit_range)
    else:
        dfa_rest = mf.compute_dfa_from_epochs(combined_rest, picks=channels, band=band, fit_range=envelope_fit_range)
        dfa_real = mf.compute_dfa_from_epochs(combined_real, picks=channels, band=band, fit_range=envelope_fit_range)
        dfa_imag = mf.compute_dfa_from_epochs(combined_imag, picks=channels, band=band, fit_range=envelope_fit_range)

    # Dataframe
    rows = []
    for ch in channels:
        rows.append({"Subject": subject_id, "Channel": ch, "Condition": "Rest", "Alpha": dfa_rest[ch]})
        rows.append({"Subject": subject_id, "Channel": ch, "Condition": "Real", "Alpha": dfa_real[ch]})
        rows.append({"Subject": subject_id, "Channel": ch, "Condition": "Imagined", "Alpha": dfa_imag[ch]})

    return pd.DataFrame(rows)


In [ ]:
df_all = pd.concat([
    compute_subject_dfa(subj, channels, band=band_range, envelope_fit_range=(30, 1000)).assign(Band=band_name)
    for band_name, band_range in bands.items()
    for subj in all_subjects
], ignore_index=True)

In [ ]:
df_real_vs_imag = df_all[df_all["Condition"].isin(["Real", "Imagined"])]

In [ ]:
plot_dfa_grouped_by_band(df_real_vs_imag, title="DFA: Real vs Imagined")